# Creating our first Pipeline

We're going to use only the non-biomedical SRs from the SYNERGY dataset. 

And we'll start with gemma3 and see where it goes!

## Dataset Preparations

In [1]:
# pkg imports
import pandas as pd

In [2]:
# our synergy datasets' paths
synergy_dataset_path = "./data/synergy_dataset/SYNERGY_combined_data.pkl"
synergy_titles_path = "./data/synergy_dataset/SYNERGY_review_titles.csv"

In [3]:
# load our datasets
syn_df = pd.read_pickle(synergy_dataset_path)
syn_titles_df = pd.read_csv(synergy_titles_path, delimiter='#', index_col='SR_id')
# a dict to map a citation_id to their title
citation2title = syn_titles_df.to_dict(orient='dict')['title']

In [4]:
# list of all the non-biomedical systematic reviews (SRs)
non_biomed_SRs = {
    "Hall_2012",
    "Radjenovic_2013",
    "Sep_2021",
    "Smid_2020"
}

In [5]:
non_biomeds_df = syn_df[syn_df['SR_id'].isin(non_biomed_SRs)]
# insert the ttitle of the systematic review in the DataFrame
non_biomeds_df.insert(2, 'SR_title', syn_df['SR_id'].apply(lambda id: citation2title[id]))
# get indexes of rows with an empty title or abstract
na_idxs = non_biomeds_df[non_biomeds_df[['title','abstract']].isna().any(axis=1)].index
# remove the rows with empty titles or abstracts
non_biomeds_df = non_biomeds_df.drop(na_idxs, axis='index')
# create a column that maps 'label_included' to a boolean 'relevant' column
non_biomeds_df['relevant'] = non_biomeds_df['label_included'].apply(bool) 
non_biomeds_df

,doi,SR_id,SR_title,title,abstract,label_included,relevant
0,https://doi.org/10.1109/indcon.2010.5712716,Hall_2012,A Systematic Literature Review on Fault Predic...,Computer vision based offset error computation...,The use of computer vision based approach has ...,0,False
1,https://doi.org/10.1109/induscon.2010.5740045,Hall_2012,A Systematic Literature Review on Fault Predic...,Design and development of a software for fault...,This paper presents an on-line fault diagnosis...,0,False
2,https://doi.org/10.1109/tpwrd.2005.848672,Hall_2012,A Systematic Literature Review on Fault Predic...,Analytical Approach to Internal Fault Simulati...,A new method for simulating faulted transforme...,0,False
3,https://doi.org/10.1109/icelmach.2008.4799852,Hall_2012,A Systematic Literature Review on Fault Predic...,Nonlinear equivalent circuit model of a tracti...,The paper presents the development of an equiv...,0,False
4,https://doi.org/10.1109/ipdps.2006.1639408,Hall_2012,A Systematic Literature Review on Fault Predic...,Fault tolerance with real-time Java,After having drawn up a state of the art on th...,0,False
...,...,...,...,...,...,...,...
66482,https://doi.org/10.1109/ictai.2010.27,Radjenovic_2013,Software fault prediction metrics: A systemati...,Attribute Selection and Imbalanced Data: Probl...,The data mining and machine learning community...,0,False
66483,https://doi.org/10.1109/acc.2001.945656,Radjenovic_2013,Software fault prediction metrics: A systemati...,Benchmarking of advanced technologies for proc...,Global competition is forcing industrial plant...,0,False
66484,https://doi.org/10.1109/icsess.2010.5552438,Radjenovic_2013,Software fault prediction metrics: A systemati...,Queueing models based performance evaluation a...,Since queueing is a common behavior in compute...,0,False
66485,https://doi.org/10.1109/wicom.2011.6040617,Radjenovic_2013,Software fault prediction metrics: A systemati...,A New Face Detection Method with GA-BP Neural ...,"In this paper, the BP neural network improved ...",0,False


## Getting DSPY going
We have our data, let's test it with DSPy!

In [6]:
# pkg imports
import dspy

/home/j/Sync/Roam/Areas/Uni/Dissertation/auto-screen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# configuring our local gemma3 model
lm = dspy.LM('ollama_chat/gemma3', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [8]:
# testing out the LM
lm("Say 'Hello world!'", temperature=0.7) 

['Hello world!\n']

## It works: basic pipeline time!
Let's create the most basic pipeline.

It just takes in the SRs titlea and a candidate citation's title and abstract.

I think this starts with a basic signature class.

In [9]:
class Relevance(dspy.Signature):
    """Classify a citation's relevance to a systematic review."""

    sr_title: str = dspy.InputField()
    citation_title: str = dspy.InputField()
    citation_abstract: str = dspy.InputField()
    relevant: bool = dspy.OutputField()
    confidence: float = dspy.OutputField()

In [10]:
first_row = non_biomeds_df.iloc[0,:]
classify = dspy.ChainOfThought(Relevance)
classify(
    sr_title=first_row['SR_title'],
    citation_title=first_row['title'],
    citation_abstract=first_row['abstract']
)

Prediction(
    reasoning='The systematic review focuses on fault prediction performance in software engineering. The citation discusses an FPGA-based computer vision system for offset error computation in *web printing machines*. While both relate to error detection and control, the citation is specifically about hardware-based image processing for a mechanical manufacturing process (printing), not software fault prediction. Therefore, the connection is weak.',
    relevant=False,
    confidence=0.3
)

## Building out the Development Set
17000 citations is still too many to run evaluations in decent time.
So, let's aim for a max of 300, that's 75 from each SR. 

In [11]:
# pkg improts
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from dspy.evaluate import Evaluate, metrics

In [12]:
# initialise our random under sampler
rus = RandomUnderSampler(random_state=42, sampling_strategy={0: 67, 1: 8})

In [13]:
def create_rus_dataset(df: pd.DataFrame, rus: RandomUnderSampler = rus) -> tuple[np.array, np.array]:
    """
    Given a group/DataFrame of a particular sytematic review.
    Returns a randomly undersampled dataset of inputs and labels.
    """
    Xs = np.array(group[['SR_title','title','abstract']])
    Ys = np.array(group['relevant'])
    X_resampled, Y_resampled = rus.fit_resample(Xs, Ys)
    return  X_resampled, Y_resampled

In [14]:
# create a development set of inputs and labels by randomly undersampling
# each systematic review so that it contains 8 positive labels and 67 negative
# ones
Xs, Ys = [], [] 
for _, group in non_biomeds_df.groupby(by='SR_id'):
    grp_Xs, grp_Ys = create_rus_dataset(group)
    Xs.append(grp_Xs)
    Ys.append(grp_Ys)
Xs = np.concatenate(Xs)
Ys = np.concatenate(Ys)

In [15]:
# create our development set of DSPy Example objects
devset = [dspy.Example(sr_title=x[0], citation_title=x[1], citation_abstract=x[2], relevant=y)\
          .with_inputs('sr_title', 'citation_title', 'citation_abstract')
          for x,y in zip(Xs, Ys)]

In [16]:
# evaluate setup
evaluator = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

In [17]:
# basic exact metric for our Relevance signature
def validate_answer(example, pred, trace=None):
    return example.relevant == pred.relevant

In [18]:
# launch evaluation with a basic 'exact match' metric
evaluator(classify, metric=validate_answer)

Average Metric: 93.00 / 300 (31.0%): 100%|████████████████████████████████████████████| 300/300 [00:06<00:00, 46.06it/s]

2025/04/18 23:52:40 INFO dspy.evaluate.evaluate: Average Metric: 93 / 300 (31.0%)


,sr_title,citation_title,citation_abstract,example_relevant,reasoning,pred_relevant,confidence,validate_answer
0,A Systematic Literature Review on Fault Prediction Performance in ...,Towards a Bayesian Approach in Modeling the Disclosure of Unique S...,Software security has both an objective and a subjective component...,False,The systematic review focuses on fault prediction performance in s...,True,0.95,
1,A Systematic Literature Review on Fault Prediction Performance in ...,Networked vehicles for automated fault detection,Creating fault detection software for complex mechatronic systems ...,False,The systematic review focuses on fault prediction performance in s...,True,0.95,
2,A Systematic Literature Review on Fault Prediction Performance in ...,Advanced Fault-Tolerant Control of Induction-Motor Drives for EV/H...,This paper describes active fault-tolerant control systems for a h...,False,The systematic review focuses on fault prediction performance in s...,False,0.60,✔️ [True]
3,A Systematic Literature Review on Fault Prediction Performance in ...,Fault ride through of DFIG wind turbines during symmetrical voltag...,Low Voltage Ride Through is an important feature for wind turbine ...,False,The systematic review focuses on fault prediction performance in s...,False,0.85,✔️ [True]
4,A Systematic Literature Review on Fault Prediction Performance in ...,Integration of Multivariate Control Charts and Neural Networks to ...,"Because of advanced technology, there are many aspects of quality ...",False,The systematic review focuses on fault prediction performance in s...,True,0.95,


np.float64(31.0)

In [19]:
# see how well basic Prediction does compared to ChainOfThought
simple_predict = dspy.Predict(Relevance)
result = evaluator(simple_predict, metric=validate_answer)

Average Metric: 36.00 / 300 (12.0%): 100%|████████████████████████████████████████████| 300/300 [00:08<00:00, 36.08it/s]


2025/04/18 23:52:48 INFO dspy.evaluate.evaluate: Average Metric: 36 / 300 (12.0%)


,sr_title,citation_title,citation_abstract,example_relevant,pred_relevant,confidence,validate_answer
0,A Systematic Literature Review on Fault Prediction Performance in ...,Towards a Bayesian Approach in Modeling the Disclosure of Unique S...,Software security has both an objective and a subjective component...,False,True,0.95,
1,A Systematic Literature Review on Fault Prediction Performance in ...,Networked vehicles for automated fault detection,Creating fault detection software for complex mechatronic systems ...,False,True,0.95,
2,A Systematic Literature Review on Fault Prediction Performance in ...,Advanced Fault-Tolerant Control of Induction-Motor Drives for EV/H...,This paper describes active fault-tolerant control systems for a h...,False,True,0.95,
3,A Systematic Literature Review on Fault Prediction Performance in ...,Fault ride through of DFIG wind turbines during symmetrical voltag...,Low Voltage Ride Through is an important feature for wind turbine ...,False,True,0.95,
4,A Systematic Literature Review on Fault Prediction Performance in ...,Integration of Multivariate Control Charts and Neural Networks to ...,"Because of advanced technology, there are many aspects of quality ...",False,True,0.95,


## New metrics
We've got a basic pipeline and evaluator going.

It's time to create the metrics we actually want to use.

We can then iterate on development and construct more sophisticated pipelines.

### F1 Score
Let's implmenet an F1 score metric.

In [20]:
# std lib imports
from collections import Counter
from typing import Callable
from math import sqrt

# pkg imports
from tqdm import tqdm

In [21]:
def precision(c: Counter) -> float:
    pos_total = c['TP'] + c['FP']
    if pos_total:
        return c['TP']/pos_total
    return float('nan')
def recall(c: Counter) -> float:
    divisor = c['TP'] + c['FN']
    if divisor:
        return c['TP']/divisor
    return float('nan')
def f1score(c: Counter) -> float:
    prec, rec = precision(c), recall(c)
    if prec and rec:
        return 2*(prec*rec)/(prec+rec)
    return float('nan')
def specificity(c: Counter) -> float:
    n = sum(c.values())
    if n:
        return 1-(c['FP']/n)
    return float('nan')

In [22]:
def validate_f1answer(example, pred, trace=None):
    if trace is None:
        if example.relevant and pred.relevant:
            return "TP" # return True Positive
        elif not example.relevant and not pred.relevant:
            return "TN" # return True Negative
        elif not example.relevant and pred.relevant:
            return "FP" # return False Positive
        else:
            return "FN" # return False Negative
    else:
        return example.relevant == pred.relevant

In [23]:
def f1evaluate(program: dspy.Program,
               devset: list[dspy.Example],
               metric: Callable = validate_f1answer) -> None:
    c = Counter()
    with tqdm(total=len(devset),
              bar_format="{postfix[0]} {postfix[1][value]:.3f} {l_bar}{bar}'| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, ' '{rate_fmt}]'",
              postfix=["F1 Score:", {'value': float('nan')}]) as t:
        for x in devset:
            pred = program(**x.inputs())
            score = metric(x, pred)
            c[score] += 1
        
            # scores
            prec = precision(c)
            rec = recall(c)
            f1 = f1score(c)
            
            # update progress bar
            t.postfix[1]['value'] = f1
            t.update()

    print(c)
    mcc = mccscore(c)
    print(f"Precision: {prec:.3f}")
    print(f"Recall: {rec:.3f}")
    print(f"F1: {f1:.3f}")
    print(f"MCC: {mcc:.3f}")
    print(f"Specificity: {specificity(c):.3f}")

In [26]:
f1evaluate(simple_predict, devset)

F1 Score: 0.195 100%|██████████████████████████████████████████████████████████'| 300/300 [00:00<00:00, ' '1696.08it/s]'

Counter({'FP': 264, 'TP': 32, 'TN': 4})
Precision: 0.108
Recall: 1.000
F1: 0.195
MCC: 0.040
Specificity: 0.120


In [27]:
f1evaluate(classify, devset)

F1 Score: 0.230 100%|██████████████████████████████████████████████████████████'| 300/300 [00:00<00:00, ' '1599.33it/s]'

Counter({'FP': 206, 'TN': 62, 'TP': 31, 'FN': 1})
Precision: 0.131
Recall: 0.969
F1: 0.230
MCC: 0.152
Specificity: 0.313


### MCC
[Pretty easy to implement](https://en.wikipedia.org/wiki/Phi_coefficient)

In [25]:
def mccscore(c: Counter) -> float:
    tp, tn, fp, fn = c['TP'], c['TN'], c['FP'], c['FN']
    dividend = tp*tn - fp*fn
    divisor = sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    if divisor:
        return dividend/divisor
    return float('nan')

## Other Pipelines
Let's try something a bit different before optimising. 

Let's see what the other built-in modules have to offer, like Refine and BestOf.

Starting with the latter, we can use the confidence parameter and see what happens.

In [234]:
def high_confidence(args, pred):
    return pred.confidence

In [241]:
best_of_3 = dspy.BestOfN(module=classify, N=3, reward_fn=high_confidence, threshold=.5)
best_of_5 = dspy.BestOfN(module=classify, N=5, reward_fn=high_confidence, threshold=.7)

In [239]:
f1evaluate(best_of_3, devset)

F1 Score: nan   1%|▍                                                               '| 2/300 [00:00<01:17, ' ' 3.87it/s]'

nan
nan


F1 Score: nan   1%|▊                                                               '| 4/300 [00:00<01:03, ' ' 4.64it/s]'

nan
nan


F1 Score: nan   2%|█▎                                                              '| 6/300 [00:01<00:59, ' ' 4.92it/s]'

...

0.21212121212121213


F1 Score: 0.218  99%|███████████████████████████████████████████████████████████▌'| 298/300 [01:08<00:00, ' ' 4.75it/s]'

0.2180451127819549


F1 Score: 0.230 100%|████████████████████████████████████████████████████████████'| 300/300 [01:08<00:00, ' ' 4.35it/s]'

0.22388059701492538
0.22962962962962963
Counter({'FP': 207, 'TN': 61, 'TP': 31, 'FN': 1})
Precision: 0.130
Recall: 0.969
F1: 0.230
MCC: 0.150


In [1]:
f1evaluate(best_of_5, devset)

NameError: name 'f1evaluate' is not defined

In [247]:
# TODO looking at implemmentation 'classify might have to be defined slightly differently
dspy.Refine(module=classify, N=3, reward_fn=high_confidence, threshold=.5)

OSError: could not get source code

In [248]:
best_of_3r = dspy.Refine(module=dspy.ChainOfThought(Relevance), N=3, reward_fn=high_confidence, threshold=.5)

OSError: could not get source code

In [ ]:
f1evaluate(best_of_3r, devset)

In [ ]:
type(f